In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

# Our application logic will be added here


In [5]:
if __name__ == "__main__":
    tf.app.run()

- Conv layer #1 : 32 5x5 filters, ReLU
- Pooling layer #1 : 2x2 filter, stride 2
- Conv layer #2 : 64 5x5, ReLU
- Pooling layer #2 : 2x2 filter, stride 2
- Dense Layer #1 : 1024, with dropout regularization rate of 0.4
- Dense layer #2(logit) : 10 neurons, one for each digit target class

- ```tf.layer``` module
    - conv2d()
    - max_pooling2d()
    - dense()

In [36]:
def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=32, 
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2],
                                   strides=2)
    
    conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5],
                            padding="same", activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], 
                                   strides=2)
    
    pool2_float = tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = tf.layers.dense(inputs=pool2_float, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(
            inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # tf.estimator.ModeKeys.TRAIN은 torch의 model.eval()이런식으로 모드 설정하는 거인듯
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        "classes" : tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    # Calculate Loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # Configure the Training Op
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # Add evaluation metrics
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## 데이터 로드

In [18]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [22]:
train_data = mnist.train.images
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images 
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

## Create Estimator

In [37]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn,
                                         model_dir="./tmp/mnist_convnet")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tmp/mnist_convnet', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12eee2128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
tensors_to_log = {"probabilities": "softmax_tensor"}

In [26]:
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

## Train model

In [38]:
%%time
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x":train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(input_fn=train_input_fn, steps=500, hooks=[logging_hook])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_convnet/model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1002 into ./tmp/mnist_convnet/model.ckpt.
INFO:tensorflow:probabilities = [[0.11338685 0.09804438 0.09890077 0.1117065  0.07914215 0.07233851
  0.08831937 0.13013196 0.11888047 0.08914905]
 [0.22139575 0.05808267 0.09177575 0.07665692 0.09406486 0.08683248
  0.08625443 0.10678002 0.10833846 0.06981861]
 [0.11615231 0.07444749 0.14457642 0.12676916 0.05408325 0.07599274
  0.1208808  0.09906986 0.10929695 0.07873105]
 [0.08268923 0.13844332 0.0864806  0.09997054 0.09561279 0.08240425
  0.09143864 0.1278395  0.11244019 0.08268098]
 [0.09177832 0.09889837 0.09478105 0.13449706 0.05284803 0.07709189
  0.08062644 0.11922047 0.11919148 0.1310668

INFO:tensorflow:loss = 1.9889755, step = 1002
INFO:tensorflow:probabilities = [[0.09253361 0.07141586 0.06976688 0.10730777 0.1009061  0.06890827
  0.08827577 0.1599896  0.08598597 0.15491024]
 [0.19353649 0.0604095  0.12752992 0.07693192 0.10280623 0.07098031
  0.09338155 0.1270849  0.0684845  0.07885456]
 [0.16129796 0.04101308 0.10871869 0.07272916 0.13347964 0.07203447
  0.12135311 0.0768422  0.1183035  0.09422812]
 [0.14961174 0.04950492 0.13218522 0.09057225 0.0772318  0.09443801
  0.20121495 0.06889219 0.06151954 0.0748293 ]
 [0.1791833  0.03768789 0.07205722 0.08555574 0.16246983 0.07734637
  0.11935106 0.08996375 0.08070178 0.09568305]
 [0.1282955  0.08294343 0.10193836 0.09405177 0.13153602 0.08741058
  0.08693719 0.06398422 0.12637374 0.0965291 ]
 [0.08205715 0.16540018 0.0939801  0.1048151  0.07773899 0.07842326
  0.09078299 0.11771542 0.10167659 0.08741021]
 [0.07710611 0.0566261  0.17402157 0.10943113 0.07616202 0.0664577
  0.20726116 0.03887387 0.11400195 0.08005843]
 [0

INFO:tensorflow:global_step/sec: 6.43241
INFO:tensorflow:probabilities = [[0.0893639  0.0999389  0.11133717 0.11374442 0.08266768 0.09531786
  0.13504618 0.06905295 0.09778672 0.10574415]
 [0.1046228  0.07931637 0.07599144 0.1518562  0.06912008 0.06276609
  0.05425243 0.18858092 0.13805926 0.07543439]
 [0.21547899 0.05836695 0.09704529 0.12109912 0.08277202 0.08432113
  0.10942725 0.05812142 0.10718579 0.06618206]
 [0.21701694 0.04630605 0.10577427 0.13773845 0.07943724 0.08191393
  0.07716581 0.06833557 0.10642882 0.07988295]
 [0.12137076 0.07745234 0.10387958 0.13483682 0.10046798 0.10171458
  0.12548849 0.05287357 0.100838   0.0810778 ]
 [0.08902385 0.0605123  0.10667322 0.10319737 0.09936167 0.08734495
  0.10360014 0.11161594 0.10485878 0.1338118 ]
 [0.12416963 0.04277426 0.04479805 0.1095593  0.06461538 0.1376782
  0.08222828 0.15731412 0.13151425 0.10534853]
 [0.1460815  0.05240452 0.06202322 0.12610616 0.09677869 0.11193623
  0.10071228 0.09970353 0.12610714 0.07814676]
 [0.1012

INFO:tensorflow:loss = 1.881695, step = 1102 (15.551 sec)
INFO:tensorflow:probabilities = [[0.08733128 0.05611915 0.12602621 0.07918689 0.13499136 0.08823878
  0.19615869 0.058467   0.07191615 0.10156453]
 [0.09973802 0.08479733 0.06438381 0.15575325 0.05828161 0.09945499
  0.06018414 0.18298739 0.09980087 0.09461861]
 [0.09152882 0.03431008 0.05328166 0.08340658 0.18110265 0.07681282
  0.12570001 0.07223534 0.11194357 0.1696785 ]
 [0.07781407 0.09558837 0.09915495 0.13702875 0.07861044 0.08514754
  0.1328453  0.08117878 0.10389744 0.10873431]
 [0.1007512  0.06665336 0.04173221 0.08581588 0.11176318 0.10040892
  0.08323745 0.1647321  0.10043462 0.1444711 ]
 [0.40149477 0.03100325 0.09568683 0.08884276 0.04059436 0.10223145
  0.08157565 0.0374529  0.07736101 0.04375688]
 [0.06914897 0.18012461 0.10787545 0.09964172 0.08231843 0.07533221
  0.08971078 0.10415712 0.10723558 0.08445504]
 [0.07710214 0.09264427 0.05926119 0.09342069 0.0572413  0.09556655
  0.06928885 0.19323729 0.11615167 0.

INFO:tensorflow:global_step/sec: 5.94882
INFO:tensorflow:probabilities = [[0.06729322 0.08565009 0.04666083 0.09673352 0.09268288 0.06707185
  0.10196683 0.16789424 0.10992366 0.16412288]
 [0.4830365  0.0239007  0.08619232 0.0655851  0.04244366 0.09110592
  0.08312478 0.03291254 0.05184646 0.03985194]
 [0.13864446 0.0372318  0.07918089 0.06924708 0.09052037 0.11343002
  0.08024444 0.10216933 0.15084045 0.13849121]
 [0.06156266 0.11033776 0.16777226 0.1701571  0.04874778 0.09004969
  0.120015   0.06431721 0.11228362 0.05475681]
 [0.07680034 0.08859752 0.10942042 0.0846585  0.10068599 0.08840089
  0.1425253  0.0929615  0.08124308 0.13470641]
 [0.08276892 0.15805578 0.10564606 0.10189303 0.11308771 0.08888461
  0.08056518 0.07259165 0.12001698 0.07648998]
 [0.18040782 0.03749428 0.06847578 0.0504136  0.10396975 0.06523751
  0.152633   0.13440259 0.07445024 0.13251543]
 [0.07273967 0.05300014 0.07533144 0.06142631 0.16366076 0.06792254
  0.12625462 0.10369962 0.11349475 0.16247016]
 [0.087

...

INFO:tensorflow:global_step/sec: 6.26865
INFO:tensorflow:probabilities = [[0.03065266 0.15774599 0.08759867 0.07938495 0.06116116 0.05332171
  0.04909429 0.21708821 0.16781099 0.09614144]
 [0.09098861 0.10742773 0.12555428 0.09633607 0.10648929 0.05975118
  0.08840214 0.07444202 0.12549183 0.12511684]
 [0.12292496 0.10479301 0.11237223 0.08958529 0.12664087 0.06239845
  0.07313968 0.15165944 0.09602985 0.06045625]
 [0.0411091  0.30668435 0.06934304 0.10385926 0.05703508 0.07485366
  0.09199688 0.08487428 0.08625762 0.08398668]
 [0.04685939 0.18456589 0.0606048  0.18190111 0.0494146  0.05673714
  0.0737082  0.13462277 0.07948029 0.13210587]
 [0.12634514 0.01373045 0.2289761  0.09503067 0.15177266 0.03652943
  0.20378196 0.03355224 0.05554481 0.05473663]
 [0.15790515 0.01074253 0.14448598 0.05747448 0.15492272 0.02786942
  0.29793492 0.02854862 0.06652058 0.05359561]
 [0.07685386 0.06521778 0.10095683 0.0754061  0.11527204 0.06195983
  0.2836431  0.05764674 0.07544686 0.08759689]
 [0.149

INFO:tensorflow:loss = 1.6173615, step = 1302 (15.948 sec)
INFO:tensorflow:probabilities = [[0.04222516 0.05092246 0.08785291 0.32746193 0.05226684 0.06193922
  0.08680569 0.06889258 0.10045051 0.12118272]
 [0.15979835 0.02366553 0.09616895 0.0700265  0.1380225  0.09657801
  0.12479728 0.06209081 0.07087347 0.1579786 ]
 [0.04907961 0.21915007 0.12156937 0.07124287 0.06333871 0.06140552
  0.04928156 0.07493469 0.20800595 0.08199165]
 [0.3463706  0.00724048 0.03122122 0.2023929  0.01994827 0.16651872
  0.04049758 0.07037914 0.07685406 0.03857699]
 [0.2189482  0.02692489 0.14728267 0.17589839 0.03429011 0.05555956
  0.13688572 0.07812188 0.10901048 0.01707809]
 [0.07547922 0.07498485 0.1026106  0.11626405 0.0989043  0.10582365
  0.12903917 0.0962466  0.07320902 0.1274385 ]
 [0.0594416  0.12059986 0.03296686 0.12313861 0.06616694 0.11913899
  0.07881848 0.15289342 0.08018903 0.16664629]
 [0.11552806 0.03728766 0.04206247 0.30328    0.0499487  0.13544527
  0.10767848 0.06029697 0.08886686 0

INFO:tensorflow:global_step/sec: 6.50825
INFO:tensorflow:probabilities = [[0.04901085 0.06867418 0.08772795 0.1244452  0.09852815 0.10742839
  0.05321185 0.05272629 0.28013015 0.07811694]
 [0.07953899 0.05923568 0.07634591 0.12077245 0.10478327 0.15154776
  0.06496412 0.0442754  0.22543855 0.07309787]
 [0.0907369  0.08182073 0.13145892 0.13327518 0.10034481 0.21354148
  0.02918428 0.04024153 0.11753888 0.06185732]
 [0.05294542 0.25247794 0.0733662  0.11482336 0.06225009 0.05245026
  0.06529301 0.1394242  0.09590321 0.09106625]
 [0.04360786 0.17645003 0.13064761 0.07444087 0.05208888 0.06601001
  0.03999646 0.07594374 0.30358055 0.03723402]
 [0.32856098 0.01253782 0.02738888 0.04675781 0.11811984 0.04709587
  0.02744741 0.24139166 0.07045715 0.08024252]
 [0.05838869 0.04938273 0.03142936 0.0443767  0.209649   0.05632784
  0.06579269 0.19583328 0.10637011 0.1824496 ]
 [0.34925848 0.01953    0.07982273 0.09089872 0.0318261  0.19178051
  0.04397153 0.07066959 0.09019958 0.03204275]
 [0.177

INFO:tensorflow:loss = 1.3390217, step = 1402 (15.366 sec)
INFO:tensorflow:probabilities = [[0.07484321 0.09825189 0.1518611  0.04553437 0.05173264 0.09798198
  0.03119426 0.09819044 0.23084098 0.11956923]
 [0.08956247 0.12977651 0.08127006 0.05251234 0.20908716 0.07697854
  0.06260126 0.0489359  0.16872612 0.08054956]
 [0.894178   0.00058416 0.02602739 0.00695057 0.00912362 0.01965311
  0.01586798 0.0065618  0.01354449 0.00750885]
 [0.13663249 0.02185358 0.06355745 0.07387172 0.19321568 0.04492304
  0.11754643 0.13179246 0.08067782 0.13592924]
 [0.05998452 0.04480715 0.02026699 0.05683232 0.21086833 0.06957967
  0.08626431 0.11525737 0.1153601  0.22077927]
 [0.07351495 0.01668235 0.07897535 0.04875476 0.28138596 0.07230458
  0.063381   0.04903893 0.12759617 0.188366  ]
 [0.1939185  0.01175798 0.03640158 0.25129476 0.03180088 0.23164387
  0.02922451 0.03517953 0.12177084 0.0570075 ]
 [0.07081387 0.01384588 0.05623921 0.06264969 0.09752344 0.05882407
  0.17031437 0.14002952 0.10317897 0

INFO:tensorflow:Saving checkpoints for 1501 into ./tmp/mnist_convnet/model.ckpt.
INFO:tensorflow:Loss for final step: 1.3285202.
CPU times: user 4min 30s, sys: 56.6 s, total: 5min 26s
Wall time: 1min 20s


In [39]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [40]:
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-23-06:54:42
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/mnist_convnet/model.ckpt-1501
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-23-06:54:47
INFO:tensorflow:Saving dict for global step 1501: accuracy = 0.7702, global_step = 1501, loss = 1.2170701
{'accuracy': 0.7702, 'loss': 1.2170701, 'global_step': 1501}
